# Persuasion Techniques in Text of Memes

## Enironment Setup

##### Disk Setup

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
# For emptying trash after each run
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')
#drive_service.files().emptyTrash().execute()

In [6]:
!pwd
!nvidia-smi

/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask2a
Tue Jan 30 20:32:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   35C    P8              26W / 350W |      3MiB / 12288MiB |      0%      Default |
|                                         |                      |                  

In [7]:
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/" #"/content/drive/MyDrive/persuasion_technique_detection/"

##### Imports

In [8]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [9]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, ViTFeatureExtractor
from sklearn.metrics import f1_score, accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
import os
from torch.nn.functional import sigmoid
from datasets import concatenate_datasets
from transformers import Trainer
from PIL import Image
import torch.nn.functional as F
from transformers import AutoModel, AutoImageProcessor,AutoTokenizer,AutoFeatureExtractor,ViTImageProcessor,ViTConfig, BertConfig, VisionTextDualEncoderConfig, VisionTextDualEncoderModel,CLIPImageProcessor

In [10]:
AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti


In [11]:
!nvidia-smi

Tue Jan 30 20:32:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   36C    P8              26W / 350W |      5MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Login to WandB

In [12]:
import wandb
import os

wandb.login(relogin=True)
#wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask2a"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/nlp-lab-ws23/.netrc


## Pre-trained Transformer Name

In [13]:
#checkpoint = "bert-base-cased"
#checkpoint="vinai/bertweet-base"
#checkpoint = "xlm-roberta-base"
#checkpoint = "xlnet-base-cased"
#checkpoint = "microsoft/deberta-v3-base"
#checkpoint = "albert-base-v2"
#checkpoint = "distilroberta-base"
#checkpoint = "microsoft/deberta-v3-base"
#checkpoint = "microsoft/deberta-v3-large"
#checkpoint= "bert-large-cased"
#checkpoint="huggyllama/llama-7b"
#checkpoint="xlm-roberta-base"
#checkpoint = "roberta-large"
#checkpoint="vinai/bertweet-large"
#checkpoint ="jayanta/twitter-roberta-base-sentiment-sentiment-memes-30epcohs"
#checkpoint = "princeton-nlp/Sheared-LLaMA-1.3B"
text_checkpoint = "vinai/bertweet-large"
img_checkpoint= "google/vit-base-patch32-224-in21k"

In [14]:
summary_dir_path = folder_name + "Summaries/summary_" + text_checkpoint.replace("/","_")+"_"+img_checkpoint.replace("/","_")+"/"

## Data Preprocessing

In [16]:
val_path="data/subtask2a/validation.json"
train_path="data/subtask2a/train.json"
test_path="data/subtask2a/dev_unlabeled.json"
en_path="data/subtask2a/english.json"
train_image_path="data/subtask2a/train_images/"
val_image_path="data/subtask2a/validation_images/"
test_image_path="data/subtask2a/dev_images/"
en_image_path= "data/subtask2a/english/"

with open(folder_name+val_path,encoding="utf8") as f:
  d = json.load(f)
  val=pd.DataFrame.from_dict(d)

with open(folder_name+train_path,encoding="utf8") as f:
  d = json.load(f)
  train=pd.DataFrame.from_dict(d)
  #mask = train_set['image'].isin(["prop_meme_24871.png"]) #also 'prop_meme_2758.png', 'prop_meme_2723.png' for texts removed
  #train_set = train_set[~mask]

with open(folder_name+test_path,encoding="utf8") as f:
  d = json.load(f)
  dev=pd.DataFrame.from_dict(d)

with open(folder_name+en_path,encoding="utf8") as f:
  d = json.load(f)
  english_set=pd.DataFrame.from_dict(d)

In [17]:
#train = pd.concat([train, dev], ignore_index=True)

In [18]:
train['labels'][0]

['Causal Oversimplification', 'Transfer', 'Flag-waving']

In [42]:
dev

,id,text,image
0,63866,ARE YOU A FAN OF THIS COUPLE?,prop_meme_3736.png
1,78505,"Hey Putin, do I look beautiful?\n\nWell Hillar...",prop_meme_22788.png
2,65750,"RUSSIAN TROOPS RETREAT VICTORIOUSLY,\nUKRAINIA...",prop_meme_13442.png
3,78716,"Want to know how to give a speech, Joe?\nWatch...",prop_meme_17567.png
4,67988,PROTECT THE RIGHTS OF INDIGENOUS PEOPLES,prop_meme_7757.png
...,...,...,...
995,68485,DEMAND KING SALMAN RELEASE CLERIC SHEIKH SALMA...,prop_meme_8031.png
996,75006,"\LISTEN TO MY ORDERS, EVIL PUPPET...\n\n... I ...",prop_meme_12805.png
997,67087,.....global confrontation that has begun is th...,prop_meme_11618.png
998,67485,IF YOU THINK THESE PUPPETS ARE THE PROBLEM\n\n...,prop_meme_5485.png


In [19]:
unique_classes = set(label for labels_list in train["labels"] for label in labels_list)
num_classes = len(unique_classes)
unique_classes

{'Appeal to (Strong) Emotions',
 'Appeal to authority',
 'Appeal to fear/prejudice',
 'Bandwagon',
 'Black-and-white Fallacy/Dictatorship',
 'Causal Oversimplification',
 'Doubt',
 'Exaggeration/Minimisation',
 'Flag-waving',
 'Glittering generalities (Virtue)',
 'Loaded Language',
 "Misrepresentation of Someone's Position (Straw Man)",
 'Name calling/Labeling',
 'Obfuscation, Intentional vagueness, Confusion',
 'Presenting Irrelevant Data (Red Herring)',
 'Reductio ad hitlerum',
 'Repetition',
 'Slogans',
 'Smears',
 'Thought-terminating cliché',
 'Transfer',
 'Whataboutism'}

In [44]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, dataset_type,image_processor,mlb):
        super().__init__()
        self.ids = list(dataset["id"])
        self.texts = list(dataset["text"])
        self.image_paths = list(dataset["image"])
        if dataset_type=="train" or dataset_type=="val" :
            self.labels = torch.tensor(mlb.transform(dataset["labels"]), dtype=torch.float32)
        self.image_processor = image_processor
        self.dataset_type = dataset_type


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if self.dataset_type=="train":
          image_path=folder_name+ train_image_path+ self.image_paths[idx]
        elif self.dataset_type=="val":
          image_path=folder_name+ val_image_path+ self.image_paths[idx]
        else:
          image_path=folder_name+ en_image_path+ self.image_paths[idx]

        image_input = self.image_processor(images=Image.open(image_path).convert("RGB"), return_tensors="pt")

        if self.dataset_type=="train" or self.dataset_type=="val":
          label = torch.tensor(self.labels[idx], dtype=torch.float32)
          return self.ids[idx],self.texts[idx],image_input,label
        else:
          return self.ids[idx],self.texts[idx],image_input

In [21]:
tokenizer = AutoTokenizer.from_pretrained(text_checkpoint)
text_model = AutoModel.from_pretrained(text_checkpoint)

# change image processor for different models
image_processor = ViTImageProcessor.from_pretrained(img_checkpoint)  #'google/vit
#image_processor = CLIPImageProcessor.from_pretrained(img_checkpoint) #"openai/clip
#image_processor = AutoImageProcessor.from_pretrained(img_checkpoint) # resnet, efficientnet
image_model = AutoModel.from_pretrained(img_checkpoint)
#image_model.config, text_model.config

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
class TextImageMultiLabelClassifier(nn.Module):
    def __init__(self, text_model, image_model,num_classes):
        super(TextImageMultiLabelClassifier, self).__init__()
        self.text_model = text_model
        self.image_model = image_model

        #image_hidden_size=image_model.config.projection_dim #openai/vit
        image_hidden_size=image_model.config.hidden_size # google/vit
        #image_hidden_size=image_model.config.hidden_sizes[-1] #resnet
        #image_hidden_size=image_model.config.hidden_dim # efficientnet

        self.lin1 = nn.Linear(text_model.config.hidden_size + image_hidden_size,512)
        self.relu=nn.ReLU()
        self.lin2=nn.Linear(512,num_classes)

    def forward(self, text_input, image_input):
        # Text encoding
        text_outputs = self.text_model(**text_input)

        text_embedding = text_outputs.last_hidden_state.mean(dim=1)

        # Image encoding
        try:
          #image_embedding=self.image_model.get_image_features(**image_input) #  "openai/clip

          image_outputs = self.image_model(**image_input)  #'google/vit
          image_embedding = image_outputs.last_hidden_state.mean(dim=1) #'google/vit

          #image_outputs = self.image_model(**image_input) # resnet, efficientnet
          #image_embedding=image_outputs.last_hidden_state # resnet, efficientnet
          #image_embedding = F.adaptive_avg_pool2d(image_embedding, (1, 1)).view(image_embedding.size(0), image_embedding.size(1)) # resnet, efficientnet
          
        except Exception as e:
          print("error:",str(e))


        # Concatenate text and image embeddings
        combined_embedding = torch.cat((text_embedding, image_embedding), dim=1)


        x = self.lin1(combined_embedding)
        x=self.relu(x)
        logits=self.lin2(x)
        return logits

In [23]:
mlb = MultiLabelBinarizer()

# Fit on the training data
mlb.fit(train['labels'])


train_dataset = CustomDataset(train,"train", image_processor,mlb)
val_dataset = CustomDataset(val,"val", image_processor,mlb)
test_dataset = CustomDataset(english_set,"english", image_processor,mlb)

batch_size=2
num_workers=2
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers,pin_memory=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers,pin_memory=True,drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers,pin_memory=True)

In [45]:
test_dataset = CustomDataset(english_set,"english", image_processor,mlb)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers,pin_memory=True)


In [24]:
train_dataset.labels[0]

tensor([0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0.])

## Training

In [25]:
transfer_learning = False
model_nodes = {
    "persuasion_or_not":None}
parent_nodes = {
    "persuasion_or_not":"persuasion_or_not"
}

In [26]:
def train():
  try:
    # Initialize a new wandb run
    wandb.init()

    # sweep agent inputs config with hyperparameters
    config = wandb.config

    learning_rate = config.learning_rate
    run_name = config.run_name+f"_{str(learning_rate)}learningRate"
    wandb.run.name = run_name

    num_epochs = 10

    text_model = AutoModel.from_pretrained(text_checkpoint)

    image_model = AutoModel.from_pretrained(img_checkpoint)

    # Instantiate the custom model
    model = TextImageMultiLabelClassifier(text_model, image_model,num_classes)
    model.cuda()
    loss_func = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    best_val_f1_macro=0.
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        all_preds = []
        all_labels = []
        print("Started training epoch:",epoch+1)
        for id, text_input, image_input, labels in train_dataloader:
            optimizer.zero_grad()
            text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)

            text_input=text_input.to("cuda")
            image_input['pixel_values']=image_input['pixel_values'].squeeze(1)
            image_input=image_input.to("cuda")
            labels=labels.to("cuda")
            logits = model(text_input, image_input)
            loss = loss_func(logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Collect predictions and labels for metrics calculation
            preds_classes = (torch.sigmoid(logits) > 0.3).int().cpu().numpy()
            all_preds.extend(preds_classes)
            all_labels.extend(labels.cpu().numpy())

        average_loss = total_loss / len(train_dataloader)

        # Calculate and log F1 score and accuracy using sklearn metrics
        f1_macro = f1_score(all_labels, all_preds,average="macro")
        f1_micro = f1_score(all_labels, all_preds,average="micro")
        accuracy = accuracy_score(all_labels, all_preds)
        # Log metrics to W&B
        wandb.log({"epoch": epoch + 1, "train_loss": average_loss, "f1_macro": f1_macro, "f1_micro":f1_micro, "train_accuracy": accuracy})

        # Validation
        model.eval()
        val_loss = 0.0
        all_val_preds = []
        all_val_labels = []
        with torch.no_grad():
            for id, text_input, image_input, labels in val_dataloader:
                text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)

                text_input=text_input.to("cuda")
                image_input['pixel_values']=image_input['pixel_values'].squeeze(1)
                image_input=image_input.to("cuda")
                labels=labels.to("cuda")
                logits = model(text_input, image_input)
                val_loss += loss_func(logits.squeeze(), labels).item()
                # Collect predictions and labels for metrics calculation
                val_preds_classes = (torch.sigmoid(logits) > 0.3).int().cpu().numpy()
                all_val_preds.extend(val_preds_classes)
                all_val_labels.extend(labels.cpu().numpy())
        average_val_loss = val_loss / len(val_dataloader)

        # Calculate and log F1 score and accuracy for validation
        val_f1_macro = f1_score(all_val_labels, all_val_preds,average="macro")
        val_f1_micro = f1_score(all_val_labels, all_val_preds,average="micro")
        val_accuracy = accuracy_score(all_val_labels, all_val_preds)
        # Log metrics to W&B
        wandb.log({"epoch": epoch + 1, "val_loss": average_val_loss, "val_f1_macro": val_f1_macro,"val_f1_micro":val_f1_micro, "val_accuracy": val_accuracy})

        # Save the model if the current validation f1_macro is better than the previous best
        if val_f1_macro > best_val_f1_macro:
          best_val_f1_macro = val_f1_macro
          best_model_state_dict = model.state_dict().copy()
          wandb.log({"eval/f1_macro":val_f1_macro})

        print(f"Epoch {epoch + 1}/{num_epochs}, Training f1-macro: {f1_macro}, Validation f1-macro: {val_f1_macro}")
        model.train()
    # Finish W&B run

    artifact = wandb.Artifact(f"best_model_{run_name}".replace("/","_"), type="model")
    artifact.add_file(folder_name+f"best_model_{run_name}.pth".replace("/","_"), torch.save(best_model_state_dict, folder_name+f"best_model_{run_name}.pth".replace("/","_")))
    wandb.run.log_artifact(artifact)
    os.remove(folder_name+f"best_model_{run_name}.pth".replace("/","_"))
    drive_service.files().emptyTrash().execute()
    wandb.finish()
  except Exception as e:
    print(f"Error in training: {str(e)}")

In [29]:
run_name=f'{text_checkpoint}-{img_checkpoint}-subtask2b'.replace("/","_")
sweep_name=f'sweep_{run_name}'
sweep_config = {
    'method': 'grid',  # can be grid, random, or bayes
    'name' : sweep_name,
    'metric': {
      'name': 'eval/f1_macro',
      'goal': 'maximize'
    },
    'parameters': {
        'learning_rate' : {
            'values': [1e-6,3e-6,5e-6,5e-5,5e-4]
        },
        'run_name': {
            'value' : run_name
        }
    }
}

# Start sweeps with specific configuration
sweep_id = wandb.sweep(sweep_config, project="subtask2a")
wandb.agent(sweep_id, train)
# Get best model of sweep
api = wandb.Api()
sweep = api.sweep(f"subtask2a/{sweep_id}")
best_run = sweep.best_run()

artifacts = best_run.logged_artifacts()

model_artifact = None
for artifact in artifacts:
    if 'model' in artifact.type:  # Adjust the condition based on your setup
        model_artifact = artifact
        break

if model_artifact != None:
  model_artifact_name = model_artifact.name
  print(f"Best Model: {model_artifact_name}")
else:
  warnings.warn(f"No models was found")

# save best model of this node
model_nodes["persuasion_or_not"] = model_artifact_name

Create sweep with ID: to0bryq0
Sweep URL: https://wandb.ai/tumnlp/subtask2a/sweeps/to0bryq0


wandb: Agent Starting Run: dza7vnov with config:
wandb: 	learning_rate: 1e-06
wandb: 	run_name: vinai_bertweet-large-google_vit-base-patch32-224-in21k-subtask2b


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Started training epoch: 1


<ipython-input-20-c028bb43c277>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float32)
<ipython-input-20-c028bb43c277>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float32)
<ipython-input-20-c028bb43c277>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float32)
<ipython-input-20-c028bb43c277>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTenso

...

<ipython-input-20-c028bb43c277>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float32)
<ipython-input-20-c028bb43c277>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float32)
<ipython-input-20-c028bb43c277>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float32)
<ipython-input-20-c028bb43c277>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTenso

Epoch 10/10, Training f1-macro: 0.03412193998073428, Validation f1-macro: 0.030863456226732314
Error in training: name 'drive_service' is not defined


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
eval/f1_macro,▁█
f1_macro,█▅▃▃▂▃▂▂▂▁
f1_micro,█▇▄▃▂▄▃▂▃▁
train_accuracy,▁▅▆▆▇▇▇▇▇█
train_loss,█▃▂▂▂▂▁▁▁▁
val_accuracy,███▁██████
val_f1_macro,▁▁▁█▁▁▂▁▁▁
val_f1_micro,▁▁▁█▁▁▁▁▁▁
val_loss,▇▃▂▂▁▄▁█▅▁
epoch,10


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
wandb: Sorting runs by -summary_metrics.eval/f1_macro


Best Model: best_model_vinai_bertweet-large-google_vit-base-patch32-224-in21k-subtask2b_5e-06learningRate:v0


In [28]:
wandb.finish()

### test on val

In [30]:
def write_json(path,data,file_name="summary.json"):
  if not isinstance(data, dict):
    data = data.to_dict("records")
  if not os.path.exists(path):
    os.makedirs(path)
  with open(path+file_name, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

In [31]:
summary = {
    "text_checkpoint" : text_checkpoint,
    "img_checkpoint":img_checkpoint,
    "model_nodes" : model_nodes,
    "train_path" : train_image_path,
    "val_path":val_image_path,
    "test_path":test_image_path
}

print(summary)

{'text_checkpoint': 'vinai/bertweet-large', 'img_checkpoint': 'google/vit-base-patch32-224-in21k', 'model_nodes': {'persuasion_or_not': 'best_model_vinai_bertweet-large-google_vit-base-patch32-224-in21k-subtask2b_5e-06learningRate:v0'}, 'train_path': 'data/subtask2a/train_images/', 'val_path': 'data/subtask2a/validation_images/', 'test_path': 'data/subtask2a/dev_images/'}


In [32]:
write_json(summary_dir_path,summary)

In [33]:
def delete_dir(path):
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"The directory {path} and all its contents have been deleted successfully")
        except OSError as e:
            print(f"Error: {e.strerror}")
    else:
        print(f"The directory {path} does not exist")

In [34]:
def delete_output_dirs(parent_directory):
    for entry in os.listdir(parent_directory):
        entry_path = os.path.join(parent_directory, entry)

        if os.path.isdir(entry_path) and entry.startswith('output_'):
            delete_dir(entry_path)

### run on test

In [38]:
api = wandb.Api()
artifact=api.artifact(model_nodes["persuasion_or_not"])
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, model_nodes["persuasion_or_not"].split(":")[0]+".pth" )
model_state_dict = torch.load(model_state_dict_path)


text_model = AutoModel.from_pretrained(text_checkpoint)
image_model = AutoModel.from_pretrained(img_checkpoint)
# Instantiate the custom model
model = TextImageMultiLabelClassifier(text_model, image_model,num_classes)
model.load_state_dict(model_state_dict)
model.cuda()

model.eval()

wandb: Downloading large artifact best_model_vinai_bertweet-large-google_vit-base-patch32-224-in21k-subtask2b_5e-06learningRate:v0, 1695.28MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.9
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TextImageMultiLabelClassifier(
  (text_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [47]:
all_preds = []

with torch.no_grad():
    for id, text_input, image_input in test_dataloader:
        text_input = tokenizer(text_input, return_tensors="pt", truncation=True, padding=True, max_length=512)
        text_input = text_input.to("cuda")
        image_input['pixel_values'] = image_input['pixel_values'].squeeze(1)
        image_input = image_input.to("cuda")
        logits = model(text_input, image_input)

        # Collect predictions and labels for metrics calculation
        test_preds_classes = (logits > 0.4).int().cpu().numpy()  # Adjust threshold as needed
        all_preds.extend(test_preds_classes)

# Convert predicted binary values to their respective labels
test_pred_labels = mlb.inverse_transform(np.array(all_preds))
# Assuming 'dev_unlabeled_set' is a DataFrame containing the test data
english_set["labels"] = test_pred_labels
english_set = english_set.drop(columns=["text", "image"])

# Write predictions to a JSON file
write_json(summary_dir_path, english_set, "english_preds.json")
